In [ ]:
from ipywidgets import Layout, Button, Box, FileUpload, Label, Checkbox, IntSlider, IntText, Button
import ipywidgets as widgets
from IPython.display import display, Image
import threading
import time

import tensorflow as tf

from keras.preprocessing import image

# uploader to image cv2
import numpy as np
import cv2
import matplotlib.pyplot as plt
 
import os

import plotly.graph_objects as go
import plotly.express as px


In [ ]:
# uploader
uploader = widgets.FileUpload(
    accept='image/*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
    )

FileUpload(value={}, accept='image/*', description='Upload')

In [ ]:
# uploader container
image_container = widgets.VBox(
    children = ()
)

In [ ]:
# wrapper
app = widgets.VBox(
    children = (uploader, image_container)
)

In [ ]:
# uploader image content and extension
def get_content_and_extension(uploader):    
    for _, value in uploader.value.items():
        return value['content'], value['metadata']['name'].split('.')[-1]

In [ ]:
# image widget from uploader
def image_widget_from_uploader(uploader):
    if uploader.value:
        file_content, file_extension = value['content'], value['metadata']['name'].split('.')[-1]
        img = widgets.Image(value=file_content, format=file_extension)
        return img

In [ ]:
# image numpy from uploader
def image_numpy_from_uploader(uploader):
    file_content, extension = value['content'], value['metadata']['name'].split('.')[-1]
    arr_from_buf = np.frombuffer(file_content, np.uint8)
    image_numpy = cv2.imdecode(arr_from_buf, cv2.IMREAD_COLOR)
    return image_numpy, extension
# plt.imshow(image_numpy)

In [ ]:
def plotly_image(image_numpy):
    fig = px.imshow(image_numpy, color_continuous_scale='gray')
    
    layout = go.Layout(
        margin=go.layout.Margin(
            l=0, #left margin
            r=0, #right margin
            b=0, #bottom margin
            t=0  #top margin
        )
    )

# Set dragmode and newshape properties; add modebar buttons
    update = fig.update_layout(layout,
                           dragmode='drawrect',
                           newshape=dict(line_color='cyan'),
#                            title_text='Drag to add annotations - use modebar to change drawing tool',
                            )

    show = fig.show(config={'modeBarButtonsToAdd':['drawline',
                                                   'drawopenpath',
                                                   'drawclosedpath',
                                                   'drawcircle',
                                                   'drawrect',
                                                   'eraseshape',
                                                  ]})

In [ ]:
image_plotly_container = widgets.VBox(
    children = ()
)

In [ ]:
# uploader
annotate_image_uploader = widgets.FileUpload(
    accept='image/*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
    )


In [ ]:
# uploader container
image_annotate_container = widgets.VBox(
    children = ()
)

In [ ]:
# image widget from uploader
def image_widget_from_annotate_uploader(annotate_image_uploader):
    if annotate_image_uploader.value:
        file_content, file_extension = value['content'], value['metadata']['name'].split('.')[-1]
        img = widgets.Image(value=file_content, format=file_extension)
        return img

In [ ]:
# image numpy from uploader
def image_numpy_from_annotate_uploader(annotate_image_uploader):
    file_content, extension = value['content'], value['metadata']['name'].split('.')[-1]
    arr_from_buf = np.frombuffer(file_content, np.uint8)
    image_numpy = cv2.imdecode(arr_from_buf, cv2.IMREAD_COLOR)
    return image_numpy, extension
# plt.imshow(image_numpy)

In [ ]:
# wrapper
app_2 = widgets.VBox(
    children = (annotate_image_uploader, image_annotate_container)
)

In [ ]:
def is_cyan(rgb):
    match = False
    if((rgb[0]>250) & (rgb[1]>250) & (rgb[2]<5)):
            match = True
    return match

In [ ]:
def img_coordinate_1(is_cyan, image_numpy):
    x1 = -1
    y1 = -1
    found = False
    
    for i in range(len(image_numpy)):
        for j in range(len(image_numpy[i])):
            if is_cyan(image_numpy[i][j]):
                x1 = i
                y1 = j
                found = True
                break
            if found:
                break
    
    return x1, y1

In [ ]:
def img_coordinate_2(is_cyan, image_numpy):
    flip_image = cv2.flip(image_numpy, -1)
    x2 = -1
    y2 = -1
    found = False
    
    for i in range(len(flip_image)):
        for j in range(len(flip_image[i])):
            if is_cyan(flip_image[i][j]):
                x2 = i
                y2 = j
                found = True
                break
            if found:
                break
    return x2, y2

In [ ]:
def img_crop(image_numpy, x1, y1, x2, y2):
    cropped_image = image_numpy[x1+4:-x2-4, y1+4:-y2-4]
    return cropped_image
#   # img_crop = widgets.Image(img_crop)
# cropped_image = plt.imshow(img_crop)
# cropped_image


In [ ]:
# image widget from image numpy
def image_bytes_crop(cropped_image, extension):
    _, encoded_image = cv2.imencode('.'+extension, img_crop,)
    image_bytes = encoded_image.tobytes()
    image_bytes_show = widgets.Image(image_bytes)
    return image_bytes_show

In [ ]:
def image_reshape(cropped_image):
    img_resize = cv2.resize(cropped_image, (256, 256))
    img_reshape_1 = img_array.reshape(-1, 256, 256, 1)
    img_reshape_1 = img_reshape_1 / 255.0
    img_reshape_2 = img_reshape_1.reshape(-1, 256, 256, 3)
    return img_reshape_2




# print(img_resize)
# print(img_resize.shape)

In [ ]:
h5_path = '/content/drive/My Drive/Inception82.h5'
model = tf.keras.models.load_model(h5_path)

In [ ]:
def predicted_image(img_reshape_2, model):
# model = model(input_shape= (32,32,1))
    result = model.predict(img_reshape_2)
    for prediction in result:
        if result == 1:
            prediction = 'Normal'
        else:
            prediction = 'Kanker'

  # prediction =  print(prediction)
    result = 'Result: ' + str(result) + ' ' + str(prediction)
    return result

In [ ]:
text_result = widgets.Label(value="Result will be shown here!")

In [ ]:
text = widgets.VBox(
    children = ([text_result]), layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    ))

In [ ]:
# uploader container
text_container = widgets.VBox(
    children = (text)
)

In [ ]:
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items = [
    Box([Label(value='Upload Mammography'), app], layout=form_item_layout),
    Box([Label(value='Annotate Image'), image_plotly_container], layout=form_item_layout),
    Box([Label(value='Upload Annotate Image'), app_2], layout=form_item_layout),
    # Box([Label(value='Cropped Image'), image_bytes_show], layout=form_item_layout),
    Box([Label(value='Result'), text_container], layout=form_item_layout)

]

form = widgets.VBox(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
  width='100%'
))
# display(form)

TraitError: ignored

In [ ]:
form

In [ ]:
# label
label_current_thread = widgets.Label()
label_thread_list = widgets.Label()

# button 
button_stop = widgets.Button(
    description = 'stop',
    icon = 'fa-stop',
    button_style = 'warning',
    layout = widgets.Layout(width='100px')
)
def button_stop_click(self):
    global thread_status
    thread_status = False
button_stop.on_click(button_stop_click)

button_check_thread = widgets.Button(
    description = 'check thread',
    button_style = '',
    layout = widgets.Layout(width='100px')
)
def button_check_thread_click(self):
    # print if work() still running
    work_count = 0
    name='work'
    for thread_object in threading.enumerate():
        if thread_object.name == name:
            work_count+=1
    if work_count > 0:
        label_thread_list.value=f'{work_count} thread with name {name} still exist. Consider use exit() and restart kernel'
    else:
        label_thread_list.value=f'No thread with name {name} exist'
button_check_thread.on_click(button_check_thread_click)

In [ ]:
# main loop
def work():

    global thread_status
    global model

    
    if thread_status:
        label_current_thread.value = 'Thread On'

    current_state =[uploader.value,
                    annotate_image_uploader.value
                    # predicted_image_container.value
                    ]
    
    while thread_status:
        
        # check if there is a change in uploader.value
        if current_state != [
            uploader.value,
            annotate_image_uploader.value
            # predicted_image_container.value
            ]:
            
            # renew state reference
            current_state = [
                uploader.value,
                annotate_image_uploader.value
                # predicted_image_container.value
                ]
            
            
            # image widget from uploader
            image_widget_raw = image_widget_from_uploader(uploader)
            
            # renew container_raw
            image_container.children = (image_widget_raw,)

            # read image numpy
            image_numpy, extension = image_numpy_from_uploader(uploader)
            
            image_plotly_container.children = plotly_image(image_numpy)

        
            image_widget_annotate = image_widget_from_annotate_uploader(annotate_image_uploader)
            
            image_annotate_container.children = (image_widget_annotate,)
            
            image_numpy, extension = image_numpy_from_annotate_uploader(annotate_image_uploader)
            
            text_result.value = predicted_image(img_reshape_2, model)
            
            if text_result.value:
                text_container.children = (text)

 
  
        # sleep for threading
        time.sleep(0.05)
        
        if not thread_status:
            label_current_thread.value = 'Thread Off'

In [ ]:
# restart flag
thread_status = False  
thread_status = True

# create thread
thread_work = threading.Thread(target=work)
thread_work.name = 'work'

# start thread
thread_work.start()

In [ ]:
form